# Device function factory 

<a href="https://colab.research.google.com/github/mark-hobbs/articles/blob/main/cuda/device-function-factory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!uv pip install -q --system numba-cuda==0.15.0

import numpy as np
from numba import  njit, prange, config
config.CUDA_ENABLE_PYNVJITLINK = 1

try:
    import google.colab
    !git clone https://github.com/mark-hobbs/articles.git
    import os
    os.chdir('articles/cuda')  # Navigate to the cuda subdirectory
except ImportError:
    pass  # Already local, no need to clone

import utils

In [2]:
np.random.seed(42)

n_elements = 1000000
alpha = np.random.randn(n_elements)
beta = np.random.randn(n_elements)
a = 1
b = 2

In [3]:
def gamma_formula(alpha, beta, a, b):
    return (alpha * a) + (beta * (1 - b))

In [4]:
def make_gamma_function(a, b):
    """
    Function factory
    """
    def gamma_function(alpha, beta):
        """
        Wrapper
        """
        return gamma_formula(alpha, beta, a, b)
    return gamma_function

In [5]:
calculate_gamma = make_gamma_function(a, b)

In [6]:
@utils.profile(runs=10)
def compute_delta(alpha, beta):
    delta = np.zeros(len(alpha))
    for i in range(len(alpha)):
        gamma = calculate_gamma(alpha[i], beta[i])
        delta[i] = gamma * (1 - gamma)
    return delta

In [7]:
delta = compute_delta(alpha, beta)
print(delta)

Function 'compute_delta' executed 10 time(s)
Average execution time: 0.2849 seconds
Min: 0.2818s, Max: 0.2896s

[ 0.22025834 -0.01704001 -0.76795334 ... -0.47248129 -0.9489488
 -3.00917413]


## Numba CUDA

In [8]:
from numba import cuda, float32

In [9]:
utils.get_cuda_device_info()

Error retrieving CUDA device information: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:
